In [ ]:
from huggingface_hub import login

# Ganti 'your_token_here' dengan token akses yang kamu salin
login('YOUR_TOKEN')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install streamlit transformers torch streamlit ngrok pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.5 MB/s eta 0:00:00


In [ ]:
import torch
torch.cuda.empty_cache()


chatbot 2

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

# Fungsi untuk memuat model dan tokenizer
@st.cache_resource
def load_model():
    model_name = "andrisusilo10/model-slang1k-Llama-2-7b"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return tokenizer, model, device

# Daftar kata-kata gaul untuk validasi
SLANG_WORDS = {
            "fomo", "baper", "yolo", "flexing", "ytta", "healing", "redflag", "followup", "fyi", "bilek", "be like", "freak", "cmiiw", "imo", "slay", "glowup", "glowing", "gamon",
            "salty", "simp", "bjir", "spill the tea", "clingy", "cegil", "ghosting", "kalcer", "jokes", "toxic", "denial", "hiddengems", "irl", "cringe", "sus", "ygy", "bucin", "famous",
            "oot", "pap", "ootd", "which is", "badmood", "trustissue", "lol", "cogil", "ovt", "relate", "chuaks", "chuak", "jujurly", "bestie", "staycation", "chuakz", "lowkey", "deeptalk",
            "tbl", "literally","sigma", "skibidi", "fanum tax", "rizz", "positif aura", "negatif aura", "gyatt", "GOAT", "mewing", "ohio", "aura level", "aura", "shees", "bussin","jokes","joke",
            "speechless","vibe", "follow","up", "hidden","gems", "deep","talk", "jokesnya", "jokenya", "fomoan", "spill","tea", "trust","issue", "glow","up", "red","flag","ate","bigl","big","l","lookmaxing",
            "look","maxing", "beta", "grinding","fanum","tax", "yapping", "yaping","npc","meme","dank","fr", "delulu", ""
}

def is_slang_text(text):
    words = set(re.findall(r'\w+', text.lower()))
    return bool(words.intersection(SLANG_WORDS))

def filter_non_slang_response(response):
    if "saya tidak tahu" in response.lower():
        return response
    return response

def translate_slang(input_text, tokenizer, model, device):
    if not is_slang_text(input_text):
        return "Maaf, saya hanya bisa menerjemahkan bahasa gaul. Mohon masukkan kalimat yang mengandung bahasa gaul."
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=200,
            num_beams=5,
            early_stopping=True,
            temperature=0.7,
            no_repeat_ngram_size=2
        )
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return filter_non_slang_response(translated_text)

def main():
    st.title("ChatBot Terjemahan Bahasa Gaul Gen Z dan Gen Alpha")

    tokenizer, model, device = load_model()

    if 'chat_history' not in st.session_state:
        st.session_state.chat_history = []

    user_input = st.text_input("Pesan...", placeholder="Masukkan kalimat gaul dari media sosial...")

    if st.button("Kirim"):
        if user_input:
            st.session_state.chat_history.append({"user": user_input})
            translated_text = translate_slang(user_input, tokenizer, model, device)

            if "Maaf, saya hanya bisa menerjemahkan bahasa gaul" in translated_text:
                response_text = translated_text
            else:
                response_text = f" Hi '{translated_text}'.\n\nApakah ini membantu?"

            st.session_state.chat_history.append({"bot": response_text})
        else:
            st.warning("Harap masukkan teks untuk diterjemahkan.")

    st.write("---")
    for chat in st.session_state.chat_history:
        if 'user' in chat:
            st.write(f"**You**: {chat['user']}")
        if 'bot' in chat:
            st.write(f"**Bot**: {chat['bot']}")

if __name__ == "__main__":
    main()


Writing app.py


In [ ]:
!ngrok config add-authtoken 2o4Rhvgfrgkauj7juFOfuNc35CA_6aAZawwLdT7QXbLL1uHZh

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import subprocess
from pyngrok import ngrok

# Memulai server Streamlit
def start_streamlit():
    command = ["streamlit", "run", "app.py", "--server.port", "8051"]
    process = subprocess.Popen(command)

# Membuat terowongan ngrok ke localhost:8501
# Use 'addr' instead of 'port' and specify the protocol (http)
public_url = ngrok.connect(addr="8051", proto="http")
print("Aplikasi Streamlit berjalan di:", public_url)

start_streamlit()

Aplikasi Streamlit berjalan di: NgrokTunnel: "https://9cbf-35-240-255-190.ngrok-free.app" -> "http://localhost:8051"
